In [13]:
import pandas
import numpy
import scipy
import string
import pprint
from string import digits
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import TweetTokenizer
from porter2stemmer import Porter2Stemmer
import nltk
from nltk.corpus import stopwords

import gensim
import os
import collections
import smart_open
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

tknzr = TweetTokenizer()
stemmer = Porter2Stemmer()
stopWords = set(stopwords.words('english'))
TOKENS = []
def readfile(fname):
    array = []
    file = open(fname, "r")
    lines = file.readlines()
    filestring = " ".join(lines)
    everything = ' '.join(filestring.split(' '))
    tokens = tknzr.tokenize(everything);
    stemmedtokens = stem(tokens)
    global TOKENS
    TOKENS = stemmedtokens
    return TOKENS

#This function takes an array and iterates over each word in it.
#If the word is not in stopwords, it stems it and appends it to a new array.
def stem(array):
    stemmed = []
    for w in array:
        if w not in stopWords:
            w = stemmer.stem(w)
            stemmed.append(w)
    return stemmed
    


In [14]:
mtarr = readfile("./marktwain/combined.txt")

In [15]:
mtstemmedwords = stem(mtarr)
len(mtstemmedwords)

254311

In [16]:
jaarr = readfile('./janeausten/ja_COMBINED_treated.txt')
jastemmedwords = stem(jaarr)
len(jastemmedwords)

292656

In [17]:
import numpy as np
jaChunksArr = np.array_split(jastemmedwords, 10000)
mtChunksArr = np.array_split(mtstemmedwords, 10000)

In [6]:
from gensim.models import doc2vec
mtStemmedArray = []
for item in mtChunksArr:
    global mtStemmedArray
    item = [item.tolist(),['marktwain']]
    mtStemmedArray.append(item)
sentenceStemmedArray = []
for item in mtStemmedArray:
    sentence = doc2vec.LabeledSentence(
    words=item[0], tags=item[1])
    sentenceStemmedArray.append(sentence)
sentenceStemmedArray

[LabeledSentence(words=['connecticut', 'yank', 'king', 'arthur', 'court', 'word', 'explan', 'warwick', 'castl', 'came', 'across', 'curious', 'strang', 'go', 'talk', 'attract', 'thr', 'thing', 'candid', 'simplic', 'marvel', 'familiar', 'ancient', 'armor', 'rest', 'compani'], tags=['marktwain']),
 LabeledSentence(words=['talk', 'fell', 'togeth', 'modest', 'peopl', 'tail', 'herd', 'shown', 'began', 'say', 'thing', 'interest', 'talk', 'along', 'soft', 'pleasant', 'flow', 'seem', 'drift', 'away', 'impercept', 'world', 'time', 'remot', 'era', 'old'], tags=['marktwain']),
 LabeledSentence(words=['forgotten', 'countri', 'gradual', 'wove', 'spell', 'seem', 'move', 'among', 'spect', 'shadow', 'dust', 'mold', 'gray', 'antiqu', 'hold', 'speech', 'relic', 'exact', 'would', 'speak', 'nearest', 'person', 'friend', 'enemi', 'familiar', 'neighbor'], tags=['marktwain']),
 LabeledSentence(words=['spoke', 'sir', 'bediv', 'sir', 'bor', 'de', 'gani', 'sir', 'launcelot', 'lake', 'sir', 'galahad', 'great', 'n

In [18]:
jaStemmedArray = []
for item in jaChunksArr:
    global jaStemmedArray
    item = [item.tolist(),['janeausten']]
    jaStemmedArray.append(item)
for item in jaStemmedArray:
    sentence = doc2vec.LabeledSentence(
    words=item[0], tags=item[1])
    sentenceStemmedArray.append(sentence)

In [19]:
combinedStemmedModel = doc2vec.Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=8)
combinedStemmedModel.build_vocab(sentenceStemmedArray)

In [ ]:
for epoch in range(10):
    combinedStemmedModel.train(sentenceStemmedArray, total_examples=combinedStemmedModel.corpus_count, epochs=combinedStemmedModel.iter)
    combinedStemmedModel.alpha -= 0.002  # decrease the learning rate
    combinedStemmedModel.min_alpha = combinedStemmedModel.alpha  # fix the learning rate, no decay

In [ ]:
len(combinedStemmedModel)

In [12]:
stemmedlist = []
for doc in sentenceStemmedArray:
    iv = combinedStemmedModel.infer_vector(doc.words)
    stemmedlist.append(iv)
len(stemmedlist)

30000

In [52]:
import pandas as pd
stemmedX = pd.DataFrame(stemmedlist)

In [55]:
yzero = np.ones(10000)
yone = np.zeros(10000)
yzero = pd.DataFrame(yzero)
yone = pd.DataFrame(yone)
Y = pd.concat([yzero,yone])
len(Y)

20000

In [73]:
from sklearn.model_selection import train_test_split
X_train_stemmed, X_test_stemmed, y_train_stemmed, y_test_stemmed = train_test_split(stemmedX, Y, test_size=0.25)
print (X_train_stemmed.shape, y_train_stemmed.shape)
print (X_test_stemmed.shape, y_test_stemmed.shape)

(15000, 100) (15000, 1)
(5000, 100) (5000, 1)


In [72]:
from sklearn.svm import SVC
rbf = SVC(C=1.0, kernel='rbf')
rbf.fit(X_train_stemmed, y_train_stemmed)
rbfStemmedPredictions = rbf.predict(X_test_stemmed)
accuracy = accuracy_score(y_test_stemmed, rbfStemmedPredictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 82.66%


In [62]:
#random forest
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=50, random_state=0)
rfc.fit(X_train_stemmed, y_train_stemmed)
rfcpredictions = rfc.predict(X_test_stemmed)
accuracy = accuracy_score(y_test_stemmed, rfcpredictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


Accuracy: 68.58%


In [78]:
#logistic regression
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
logreg = linear_model.LogisticRegression()
logreg.fit(X_train_stemmed, y_train_stemmed)
logregpredictions = logreg.predict(X_test_stemmed)
accuracy = accuracy_score(y_test_stemmed, logregpredictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 83.70%


In [79]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation='relu', solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(5, 2), random_state=1)
mlp.fit(X_train_stemmed, y_train_stemmed)
mlppredictions = mlp.predict(X_test_stemmed)
accuracy = accuracy_score(y_test_stemmed, mlppredictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 49.30%


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
